In [1]:
from Python.tigre.utilities.io.VarianDataLoader import (
    VarianDataLoader
)
from Python.tigre.utilities.io.varian.varian_io import (
    ScanParams
)
from Python.tigre.algorithms.single_pass_algorithms import FDK
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mydir = "C:/CBCT_Scatter_Removal_Project/data/clinical/4100758/2021-07-06_101037/11528a87-b894-4239-846d-45d8fd1efeda"

In [ ]:
log_projs,geometry,angles = VarianDataLoader(mydir,sc=1,dps=1)

In [ ]:
plt.imshow(log_projs[0])
plt.colorbar()

In [ ]:
np.save('./data/proj_sks.npy',log_projs)


In [ ]:
recon=FDK(log_projs,geometry,angles,verbose=True)

In [ ]:
np.save('./data/recon_sks.npy',recon)

In [ ]:
recon[recon<0]=0

In [ ]:
from Python.tigre.utilities.mask_CBCT import maskCBCT

In [ ]:
plt.imshow(maskCBCT(recon)[60],cmap='gray')
plt.colorbar()